<a href="https://colab.research.google.com/github/kswam/Fake-News-Detection/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re


In [25]:
dataset = pd.read_csv("/content/drive/MyDrive/fake news/train.csv")
test_dataset = pd.read_csv("/content/drive/MyDrive/fake news/test.csv")

In [26]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [27]:
test_dataset.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [28]:
dataset.shape

(20800, 5)

In [29]:
print(dataset.isna().sum())
print(test_dataset.isna().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
id          0
title     122
author    503
text        7
dtype: int64


In [30]:
dataset = dataset.dropna(subset = ["text"])
test_dataset = test_dataset.dropna(subset = ["text"])
print(dataset.isna().sum())
print(test_dataset.isna().sum())

id           0
title      558
author    1918
text         0
label        0
dtype: int64
id          0
title     122
author    496
text        0
dtype: int64


In [31]:
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [32]:
def handle_nan(train_data,test_data):
    '''Input: Data to the function containing Nan values.
       Output : Cleaned data containing no Nan values.
       Function: Cleaning Nan values.
     '''
    dataset = train_data.fillna(" ")
    test_dataset  = test_data.fillna(" ")
    return dataset,test_dataset

dataset,test_dataset = handle_nan(dataset,test_dataset)

In [33]:
dataset["merged"] = dataset["title"]+" "+dataset["author"]
test_dataset["merged"] = test_dataset["title"]+" "+test_dataset["author"]

In [34]:
X = dataset.drop(columns =["label"],axis=1)
y = dataset["label"].values

y.shape
y = y.reshape(-1,1)
print(X.shape)
print(y.shape)

(20761, 5)
(20761, 1)


In [35]:
pre_data = X.copy()
pre_data.reset_index(inplace = True)
pre_data_test = test_dataset.copy()
pre_data_test.reset_index(inplace = True)

In [36]:
#preprocessing text 
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
set_stopwords = set(stopwords.words("english"))
set_stopwords.discard("no")
set_stopwords.discard("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
#stemming the words
import nltk
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [38]:
def perform_preprocess(data):
    '''Input: Data to be processed
       Output: Preprocessed data
    '''
    corpus = []
    for i in range(0,len(data)):
        test = re.sub('[^a-zA-Z]',' ',str(data['merged'][i]))
        test = test.lower()
        test = test.split()
        test = [porter.stem(word) for word in test if word not in stopwords.words('english')]
        test = ' '.join(test)
        corpus.append(test)
    return corpus

In [39]:
train_corpus = perform_preprocess(pre_data)
test_corpus  = perform_preprocess(pre_data_test)
train_corpus[1]

'flynn hillari clinton big woman campu breitbart daniel j flynn'

In [40]:
test_corpus[1]

'russian warship readi strike terrorist near aleppo'

In [41]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer_train = tokenizer.fit_on_texts(train_corpus)
tokenizer_test = tokenizer.fit_on_texts(test_corpus)
sequences_train = tokenizer.texts_to_sequences(train_corpus)
sequences_test = tokenizer.texts_to_sequences(test_corpus)



In [42]:
#embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
given_length = 20
embedd_train = pad_sequences(sequences_train,padding='pre',maxlen=given_length)
embedd_test  = pad_sequences(sequences_test,padding='pre',maxlen=given_length)
print(embedd_train)

[[   0    0    0 ...  513 3547 3767]
 [   0    0    0 ...   47   87  842]
 [   0    0    0 ...  110  631   17]
 ...
 [   0    0    0 ... 1968  482 1734]
 [   0    0    0 ... 4741  202  476]
 [   0    0    0 ... 1729   42 2040]]


In [43]:
print(embedd_test)

[[    0     0     0 ...     2    42  3532]
 [    0     0     0 ...   282   530   266]
 [    0     0     0 ...    44  1048   960]
 ...
 [    0     0     0 ...     2    66   811]
 [    0     0     0 ...    53   127  3518]
 [    0     0     0 ...     2 18712  4017]]


In [44]:
# Converting Embedding repr. to array
x_final = np.array(embedd_train)
y_final = np.array(y)
x_test_final = np.array(embedd_test)

In [45]:
# Dimensions of prev. array repr.
x_final.shape,y_final.shape,x_test_final.shape

((20761, 20), (20761, 1), (5193, 20))

In [46]:
#train and split 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size = 0.1,stratify=y_final, random_state=2022)
X_train,x_valid,Y_train,y_valid = train_test_split(x_train,y_train,test_size = 0.1,random_state=2022,stratify = y_train)
x_test_final = x_test_final

In [47]:
#logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
MODEL_1 = LogisticRegression(max_iter = 800)
MODEL_1.fit(X_train,Y_train)
PRED_1 = MODEL_1.predict(x_test)
CR_1 = classification_report(y_test,PRED_1)
print(CR_1)

              precision    recall  f1-score   support

           0       0.74      0.70      0.72      1039
           1       0.71      0.75      0.73      1038

    accuracy                           0.72      2077
   macro avg       0.73      0.72      0.72      2077
weighted avg       0.73      0.72      0.72      2077



/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
#Naive bayers - Multinomial NB
from sklearn.naive_bayes import MultinomialNB
MODEL_2 = MultinomialNB()
MODEL_2.fit(X_train,Y_train)
PRED_2 = MODEL_2.predict(x_test)
CR_2 = classification_report(y_test,PRED_2)
print(CR_2)


              precision    recall  f1-score   support

           0       0.74      0.71      0.72      1039
           1       0.72      0.75      0.73      1038

    accuracy                           0.73      2077
   macro avg       0.73      0.73      0.73      2077
weighted avg       0.73      0.73      0.73      2077



/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
MODEL_3 = RandomForestClassifier()
MODEL_3.fit(X_train,Y_train)
PRED_3 = MODEL_3.predict(x_test)
CR_3 = classification_report(y_test,PRED_3)
print(CR_3)

<ipython-input-49-6be69fcf60e2>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  MODEL_3.fit(X_train,Y_train)


              precision    recall  f1-score   support

           0       0.99      0.94      0.96      1039
           1       0.94      0.99      0.97      1038

    accuracy                           0.96      2077
   macro avg       0.97      0.96      0.96      2077
weighted avg       0.97      0.96      0.96      2077



In [50]:
#SVC
from sklearn.svm import SVC
MODEL_4 = SVC()
MODEL_4.fit(X_train,Y_train)
PRED_4 = MODEL_4.predict(x_test)
CR_4 = classification_report(y_test,PRED_4)
print(CR_4)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.77      0.82      0.79      1039
           1       0.81      0.75      0.78      1038

    accuracy                           0.79      2077
   macro avg       0.79      0.79      0.79      2077
weighted avg       0.79      0.79      0.79      2077



In [51]:
#DecisionTress Classifier
from sklearn.tree import DecisionTreeClassifier
MODEL_5 = DecisionTreeClassifier()
MODEL_5.fit(X_train,Y_train)
PRED_5 = MODEL_5.predict(x_test)
CR_5 = classification_report(y_test,PRED_5)
print(CR_5)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1039
           1       0.97      0.97      0.97      1038

    accuracy                           0.97      2077
   macro avg       0.97      0.97      0.97      2077
weighted avg       0.97      0.97      0.97      2077



In [53]:
Score_1 = accuracy_score(y_test,PRED_1)
Score_2 = accuracy_score(y_test,PRED_2)
Score_3 = accuracy_score(y_test,PRED_3)
Score_4 = accuracy_score(y_test,PRED_4)
Score_5 = accuracy_score(y_test,PRED_5)

results = pd.DataFrame([["Logistic Regression",Score_1],["Naive Bayes-MultinomialNB",Score_2],["Random Forest Classifier",Score_3],
                       ["SVC",Score_4],["Decision Tree Classifier",Score_5]], columns=["Model","Accuracy"])
results

,Model,Accuracy
0,Logistic Regression,0.724603
1,Naive Bayes-MultinomialNB,0.727973
2,Random Forest Classifier,0.964853
3,SVC,0.786712
4,Decision Tree Classifier,0.970631


In [54]:
predictions_test = pd.DataFrame(MODEL_5.predict(x_test_final))
test_id = pd.DataFrame(test_dataset["id"])
submission = pd.concat([test_id,predictions_test],axis=1)
submission.columns = ["id","label"]
submission.to_csv("Submission.csv",index=False)

submission.head()

,id,label
0,20800.0,0.0
1,20801.0,1.0
2,20802.0,1.0
3,20803.0,0.0
4,20804.0,1.0
